<a href="https://colab.research.google.com/github/thislis/textdenoise/blob/main/%ED%83%90%EB%85%BC_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import shutil
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
lines = pd.read_csv('/content/drive/MyDrive/단어만 8만개.csv', names=['A', 'B'], sep=',')
print(lines)

           A      B
0       가와가현   가가와현
1       가대가소   가가대소
2       가문가전   가가문전
3      가붓자가식  가가붓자식
4       가성가자   가가성자
...      ...    ...
81390   힘이받천   힘받이천
81391   힘이받침   힘받이침
81392   힘잇받감   힘받잇감
81393   힝항뚱뚱   힝뚱항뚱
81394   힝힝뚱뚱   힝뚱힝뚱

[81395 rows x 2 columns]


In [4]:
lines = lines.loc[:, 'A':'B']
lines = lines[0:80000]
lines.sample(10)

,A,B
10995,길이잡축,길잡이축
63896,정부리사,정리부사
6129,골횟판대,골판횟대
6521,공편연대,공연편대
78117,해랭파이,해파랭이
3063,거기르병,거르기병
38372,사가생판,사생가판
48381,아머이어니,아이어머니
26334,명방대이록,명이대방록
79318,호식회물,호회식물


In [5]:
lines.B = lines.B.apply(lambda x : '\t '+ str(x) + ' \n')
lines.sample(10)

,A,B
43678,송헤장엄,\t 송장헤엄 \n
57780,유무형적,\t 유형무적 \n
17157,대탕구반,\t 대구탕반 \n
35683,불의수근,\t 불수의근 \n
4766,계중산심,\t 계산중심 \n
74476,특시별도,\t 특별시도 \n
33066,벼같락이,\t 벼락같이 \n
51768,엉진망창,\t 엉망진창 \n
76330,풍전치체,\t 풍치전체 \n
49187,알이낳철,\t 알낳이철 \n


In [6]:
# 글자 집합 구축
src_vocab=set()
for l in lines.A: # 1줄씩 읽음
  for char in str(l): # 1개의 글자씩 읽음
    src_vocab.add(char)

tar_vocab=set()
for l in lines.B:
  for char in l:
     tar_vocab.add(char)

In [7]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

1684
1687


In [8]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

In [9]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])

In [10]:
encoder_input = []
for line in lines.A: #입력 데이터에서 1줄씩 단어을 읽음
    temp_X = []
    for w in str(line): #각 줄에서 1개씩 글자를 읽음
      temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X)
print(encoder_input[:5])

[[2, 1053, 2, 1624], [2, 314, 2, 846], [2, 621, 2, 1152], [2, 720, 1128, 2, 903], [2, 828, 2, 1128]]


In [11]:
decoder_input = []
for line in lines.B:
    temp_X = []
    for w in line:
      temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])

[[1, 3, 5, 5, 1056, 1627, 3, 2], [1, 3, 5, 5, 317, 849, 3, 2], [1, 3, 5, 5, 624, 1155, 3, 2], [1, 3, 5, 5, 723, 1131, 906, 3, 2], [1, 3, 5, 5, 831, 1131, 3, 2]]


In [12]:
decoder_target = []
for line in lines.B:
    t=0
    temp_X = []
    for w in line:
      if t>0:
        temp_X.append(tar_to_index[w])
      t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[3, 5, 5, 1056, 1627, 3, 2], [3, 5, 5, 317, 849, 3, 2], [3, 5, 5, 624, 1155, 3, 2], [3, 5, 5, 723, 1131, 906, 3, 2], [3, 5, 5, 831, 1131, 3, 2]]


In [13]:
max_src_len = max([len(line) for line in lines.A])
max_tar_len = max([len(line) for line in lines.B])

lenlineA = []
lenlineB = []

for line in lines.A:
  lenlineA.append(len(str(line)))

for line in lines.B:
  lenlineB.append(len(str(line)))

In [14]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [15]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [16]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [17]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

In [18]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 함.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [19]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=512, epochs=50, validation_split=0.2)

Epoch 1/50
125/125 [==============================] - 13s 70ms/step - loss: 3.4530 - val_loss: 3.2476
Epoch 2/50
125/125 [==============================] - 6s 45ms/step - loss: 2.9050 - val_loss: 3.2365
Epoch 3/50
125/125 [==============================] - 6s 45ms/step - loss: 2.7468 - val_loss: 3.1279
Epoch 4/50
125/125 [==============================] - 6s 45ms/step - loss: 2.5290 - val_loss: 2.8737
Epoch 5/50
125/125 [==============================] - 6s 46ms/step - loss: 2.2584 - val_loss: 2.6272
Epoch 6/50
125/125 [==============================] - 6s 46ms/step - loss: 1.9486 - val_loss: 2.3112
Epoch 7/50
125/125 [==============================] - 6s 46ms/step - loss: 1.5916 - val_loss: 2.0009
Epoch 8/50
125/125 [==============================] - 6s 45ms/step - loss: 1.2447 - val_loss: 1.7033
Epoch 9/50
125/125 [==============================] - 6s 45ms/step - loss: 0.9501 - val_loss: 1.4953
Epoch 10/50
125/125 [==============================] - 6s 45ms/step - loss: 0.7264 - val_l

In [20]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [21]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [22]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [23]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        # output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        datalist = decoder_model.predict([target_seq] + states_value)
        output_tokens = datalist[0]
        h = datalist[1]
        c = datalist[2]

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index + 1]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [24]:
for seq_index in [1,2,3,10]: # 입력 단어의 인덱스
    input_seq = encoder_input[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentence = decoded_sentence.replace("ㅅ", "")
    decoded_sentence = decoded_sentence.replace("\t", "")
    print(35 * "-")
    print('입력 단어:', lines.A[seq_index])
    print('정답 단어:', lines.B[seq_index][1:len(lines.B[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 단어:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 단어: 가대가소
정답 단어:  가가대소 
번역기가 번역한 단어: 각각댁속
-----------------------------------
입력 단어: 가문가전
정답 단어:  가가문전 
번역기가 번역한 단어: 각각묻절
-----------------------------------
입력 단어: 가붓자가식
정답 단어:  가가붓자식 
번역기가 번역한 단어: 각각붕작신
-----------------------------------
입력 단어: 가역감관
정답 단어:  가감역관 
번역기가 번역한 단어: 각갑엮괄
